#Mariz Essam Sobhy Ghaly
#1808421

In \[1\]:

    import os
    import random
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

    import tensorflow as tf
    from keras.utils.vis_utils import plot_model
    from keras.utils.np_utils import to_categorical
    from keras.preprocessing.image import ImageDataGenerator
    from keras.models import Sequential
    from keras.layers import Conv2D, AveragePooling2D, Dense, Flatten
    from keras.callbacks import LearningRateScheduler, EarlyStopping

    BATCH_SIZE = 4096

In \[2\]:

    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)                    
    random.seed(seed)

# Reading Data and pre-processing<a href="#Reading-Data-and-pre-processing" class="anchor-link">¶</a>

In \[3\]:

    train = pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_train.csv")
    test = pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_test.csv")

In \[4\]:

    train.describe()

Out\[4\]:

|       | label        | pixel1       | pixel2       | pixel3       | pixel4       | pixel5       | pixel6       | pixel7       | pixel8       | pixel9       | ... | pixel775     | pixel776     | pixel777     | pixel778     | pixel779     | pixel780     | pixel781     | pixel782     | pixel783     | pixel784    |
|-------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|-----|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|--------------|-------------|
| count | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | ... | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.000000 | 60000.00000 |
| mean  | 4.500000     | 0.000900     | 0.006150     | 0.035333     | 0.101933     | 0.247967     | 0.411467     | 0.805767     | 2.198283     | 5.682000     | ... | 34.625400    | 23.300683    | 16.588267    | 17.869433    | 22.814817    | 17.911483    | 8.520633     | 2.753300     | 0.855517     | 0.07025     |
| std   | 2.872305     | 0.094689     | 0.271011     | 1.222324     | 2.452871     | 4.306912     | 5.836188     | 8.215169     | 14.093378    | 23.819481    | ... | 57.545242    | 48.854427    | 41.979611    | 43.966032    | 51.830477    | 45.149388    | 29.614859    | 17.397652    | 9.356960     | 2.12587     |
| min   | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | ... | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.00000     |
| 25%   | 2.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | ... | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.00000     |
| 50%   | 4.500000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | ... | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.00000     |
| 75%   | 7.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | ... | 58.000000    | 9.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.000000     | 0.00000     |
| max   | 9.000000     | 16.000000    | 36.000000    | 226.000000   | 164.000000   | 227.000000   | 230.000000   | 224.000000   | 255.000000   | 254.000000   | ... | 255.000000   | 255.000000   | 255.000000   | 255.000000   | 255.000000   | 255.000000   | 255.000000   | 255.000000   | 255.000000   | 170.00000   |

8 rows × 785 columns

In \[5\]:

    train['label'].describe()

Out\[5\]:

    count    60000.000000
    mean         4.500000
    std          2.872305
    min          0.000000
    25%          2.000000
    50%          4.500000
    75%          7.000000
    max          9.000000
    Name: label, dtype: float64

## Labels Freq. Analysis<a href="#Labels-Freq.-Analysis" class="anchor-link">¶</a>

In \[6\]:

    def num_to_cloth(argument):
        switcher = {
            0: "T-shirt/top",
            1: "Trouser",
            2: "Pullover",
            3: "Dress",
            4: "Coat",
            5: "Sandal",
            6: "Shirt",
            7: "Sneaker",
            8: "Bag",
            9: "Ankle boot"
        }
        return switcher.get(argument, "nothing")

In \[7\]:

    elements_count = {}
    for element in train["label"]:
       if element in elements_count:
          elements_count[element] += 1
       else:
          elements_count[element] = 1
    print(type(elements_count))
    for key, value in sorted(elements_count.items()):
       print(f"{key}: {value} => {num_to_cloth(key)}")

    print("\nWe have {} cloth types and each one repeated exactly 6000 times".format(len(elements_count)))

    <class 'dict'>
    0: 6000 => T-shirt/top
    1: 6000 => Trouser
    2: 6000 => Pullover
    3: 6000 => Dress
    4: 6000 => Coat
    5: 6000 => Sandal
    6: 6000 => Shirt
    7: 6000 => Sneaker
    8: 6000 => Bag
    9: 6000 => Ankle boot

    We have 10 cloth types and each one repeated exactly 6000 times

In \[9\]:

    train.label.value_counts()

Out\[9\]:

    2    6000
    9    6000
    6    6000
    0    6000
    3    6000
    4    6000
    5    6000
    8    6000
    7    6000
    1    6000
    Name: label, dtype: int64

In \[10\]:

    train.head()

Out\[10\]:

|     | label | pixel1 | pixel2 | pixel3 | pixel4 | pixel5 | pixel6 | pixel7 | pixel8 | pixel9 | ... | pixel775 | pixel776 | pixel777 | pixel778 | pixel779 | pixel780 | pixel781 | pixel782 | pixel783 | pixel784 |
|-----|-------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|
| 0   | 2     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | ... | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        |
| 1   | 9     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | ... | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        |
| 2   | 6     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 5      | 0      | ... | 0        | 0        | 0        | 30       | 43       | 0        | 0        | 0        | 0        | 0        |
| 3   | 0     | 0      | 0      | 0      | 1      | 2      | 0      | 0      | 0      | 0      | ... | 3        | 0        | 0        | 0        | 0        | 1        | 0        | 0        | 0        | 0        |
| 4   | 3     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | ... | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        |

5 rows × 785 columns

In \[11\]:

    test.head()

Out\[11\]:

|     | label | pixel1 | pixel2 | pixel3 | pixel4 | pixel5 | pixel6 | pixel7 | pixel8 | pixel9 | ... | pixel775 | pixel776 | pixel777 | pixel778 | pixel779 | pixel780 | pixel781 | pixel782 | pixel783 | pixel784 |
|-----|-------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|
| 0   | 0     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 9      | 8      | ... | 103      | 87       | 56       | 0        | 0        | 0        | 0        | 0        | 0        | 0        |
| 1   | 1     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | ... | 34       | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        |
| 2   | 2     | 0      | 0      | 0      | 0      | 0      | 0      | 14     | 53     | 99     | ... | 0        | 0        | 0        | 0        | 63       | 53       | 31       | 0        | 0        | 0        |
| 3   | 2     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | ... | 137      | 126      | 140      | 0        | 133      | 224      | 222      | 56       | 0        | 0        |
| 4   | 3     | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | 0      | ... | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        | 0        |

5 rows × 785 columns

In \[13\]:

    print(train.shape)
    print(test.shape)

    (60000, 785)
    (10000, 785)

In \[14\]:

    print(f"Training observations {train.shape[0]}, Test observations {test.shape[0]} \n")

    Training observations 60000, Test observations 10000 

## Cleaning the data from NULL or Duplicates<a href="#Cleaning-the-data-from-NULL-or-Duplicates" class="anchor-link">¶</a>

In \[15\]:

    #If the output is zero, it means that there are no missing values in our dataset.
    print(train.isnull().sum().sum())

    0

In \[16\]:

    # dropping duplicates by considering all rows
    count = 0
    for element in train.duplicated():
        count = count + 1 if (element == True) else count
    print("number of duplicate elements is", count)

    number of duplicate elements is 43

In \[21\]:

    # dropping duplicates by considering all columns other than ID & Species
    pixels = list(train.columns)[1:]
    print(train.shape)
    train.drop_duplicates(subset=pixels, inplace=True)
    print(train.shape)

    (60000, 785)
    (59957, 785)

### Conclusion: The Data contains no missing values but contains 43 duplicate element<a href="#Conclusion:-The-Data-contains-no-missing-values-but-contains-43-duplicate-element" class="anchor-link">¶</a>

## Visualizing some of the images<a href="#Visualizing-some-of-the-images" class="anchor-link">¶</a>

In \[22\]:

    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    plt.figure(figsize=(12,9))
    for i in range(0,12):
        plt.subplot(3,4,i+1)
        image_resized = np.resize(train.iloc[i,1:].values,(28,28))
        plt.title(class_names[train.iloc[i,0]])
        plt.imshow(image_resized, cmap='gray', interpolation='none')
        plt.axis('off')

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/031ac33eedbd4d17fe607093fe2135bfbf0f7977.png" class="jp-needs-light-background" />

### Extracting Features and Labels from the data<a href="#Extracting-Features-and-Labels-from-the-data" class="anchor-link">¶</a>

In \[23\]:

    X_train = np.array(train.iloc[:, 1:])
    y_train = to_categorical(np.array(train.iloc[:, 0]))

In \[24\]:

    y_train

Out\[24\]:

    array([[0., 0., 1., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 1.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 1., 0.],
           [0., 0., 0., ..., 0., 1., 0.],
           [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In \[25\]:

    X_test = np.array(test.iloc[:, 1:])
    y_test = to_categorical(np.array(test.iloc[:, 0]))

In \[26\]:

    y_test

Out\[26\]:

    array([[1., 0., 0., ..., 0., 0., 0.],
           [0., 1., 0., ..., 0., 0., 0.],
           [0., 0., 1., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 1., 0.],
           [0., 0., 0., ..., 0., 1., 0.],
           [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

## Correlation Analysis<a href="#Correlation-Analysis" class="anchor-link">¶</a>

In \[31\]:

    corr_matrix = train.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
    upper_tri.head()

    /opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
    Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
      

Out\[31\]:

|        | label | pixel1   | pixel2   | pixel3   | pixel4   | pixel5   | pixel6   | pixel7   | pixel8   | pixel9   | ... | pixel775 | pixel776 | pixel777 | pixel778 | pixel779 | pixel780 | pixel781 | pixel782 | pixel783 | pixel784 |
|--------|-------|----------|----------|----------|----------|----------|----------|----------|----------|----------|-----|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|
| label  | NaN   | 0.000676 | 0.002942 | 0.010453 | 0.007119 | 0.004629 | 0.011058 | 0.036828 | 0.085385 | 0.161761 | ... | 0.362817 | 0.258533 | 0.183235 | 0.090386 | 0.077161 | 0.066846 | 0.018054 | 0.045588 | 0.059963 | 0.021773 |
| pixel1 | NaN   | NaN      | 0.297899 | 0.067551 | 0.046607 | 0.026630 | 0.026172 | 0.012096 | 0.012225 | 0.009644 | ... | 0.000641 | 0.004628 | 0.004609 | 0.000975 | 0.002305 | 0.002442 | 0.000109 | 0.008764 | 0.026388 | 0.041581 |
| pixel2 | NaN   | NaN      | NaN      | 0.575033 | 0.138709 | 0.054353 | 0.033184 | 0.022766 | 0.017138 | 0.016821 | ... | 0.000494 | 0.004861 | 0.006817 | 0.002097 | 0.004427 | 0.002341 | 0.004271 | 0.014215 | 0.021296 | 0.022161 |
| pixel3 | NaN   | NaN      | NaN      | NaN      | 0.387468 | 0.118136 | 0.087300 | 0.060937 | 0.035942 | 0.029674 | ... | 0.010095 | 0.016706 | 0.018323 | 0.006163 | 0.003821 | 0.001494 | 0.006861 | 0.013151 | 0.009946 | 0.015657 |
| pixel4 | NaN   | NaN      | NaN      | NaN      | NaN      | 0.573172 | 0.325683 | 0.242987 | 0.141033 | 0.085302 | ... | 0.009690 | 0.018710 | 0.023391 | 0.016759 | 0.009700 | 0.010101 | 0.023940 | 0.012388 | 0.003072 | 0.008422 |

5 rows × 785 columns

In \[32\]:

    im_rows, im_cols = 28, 28
    input_shape = (im_rows, im_cols, 1)

    # train and validate sets
    X_train = X_train.reshape(X_train.shape[0], im_rows, im_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], im_rows, im_cols, 1)

    # normalisation
    X_train = X_train/255
    X_test = X_test/255

    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)

    X_train shape: (59957, 28, 28, 1)
    X_test shape: (10000, 28, 28, 1)
    y_train shape: (59957, 10)
    y_test shape: (10000, 10)

In \[33\]:

    def lenet5(input_sh, opt=tf.keras.optimizers.Adam(learning_rate=0.001)):
        
        model = Sequential(
            [
                Conv2D(filters = 6, kernel_size = (5, 5), strides = (1, 1), activation = "tanh", input_shape = input_sh),
                AveragePooling2D(pool_size = (2, 2)),
                Conv2D(filters = 16, kernel_size = (5, 5), strides = (1, 1), activation = "tanh"),
                AveragePooling2D(pool_size = (2, 2), strides = 2),
                
                Flatten(),
                Dense(units = 120, activation = "tanh"),
                Dense(units = 84, activation = "tanh"),
                Dense(units = 10, activation = "softmax")
            ]
        )
        
        model.compile(optimizer=opt,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
        model.summary()
        
        return model

In \[46\]:

    model_1 = lenet5(input_shape, tf.keras.optimizers.Adam(learning_rate=0.001))

    Model: "sequential_2"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    conv2d_4 (Conv2D)            (None, 24, 24, 6)         156       
    _________________________________________________________________
    average_pooling2d_4 (Average (None, 12, 12, 6)         0         
    _________________________________________________________________
    conv2d_5 (Conv2D)            (None, 8, 8, 16)          2416      
    _________________________________________________________________
    average_pooling2d_5 (Average (None, 4, 4, 16)          0         
    _________________________________________________________________
    flatten_2 (Flatten)          (None, 256)               0         
    _________________________________________________________________
    dense_6 (Dense)              (None, 120)               30840     
    _________________________________________________________________
    dense_7 (Dense)              (None, 84)                10164     
    _________________________________________________________________
    dense_8 (Dense)              (None, 10)                850       
    =================================================================
    Total params: 44,426
    Trainable params: 44,426
    Non-trainable params: 0
    _________________________________________________________________

In \[47\]:

    from sklearn.model_selection import KFold
    from sklearn.metrics import confusion_matrix
    k = 5
    cross_val = KFold(k, shuffle=True, random_state=1)
    fold_count = 1

    # For training epochs
    epochs = 32

    # For loss & acc plotting
    histories = []

    # For testing/evaluation acc scores
    eval_scores = []

    # For callbacks
    es_callbacks = EarlyStopping(monitor="val_loss",
                                              mode="min",
                                              verbose=1,
                                              patience=4)

In \[48\]:

    for train, validation in cross_val.split(X_train):
        print("="*80)
        print("Fold-{}".format(fold_count))
        print("-"*80)
        print("Training & Validation")
        fold_count = fold_count + 1
        
        
        X_train_, y_train_ = X_train[train], y_train[train]
        X_val, y_val = X_train[validation], y_train[validation]
        
        history = model_1.fit(X_train_, y_train_,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            callbacks=[es_callbacks])
        
        print("-"*80)
        print("Testing/evaluation")
        eval_loss, eval_accuracy = model_1.evaluate(X_test, y_test)
        
        histories.append(history)
        eval_scores.append(eval_accuracy)
        print("_"*80)

    ================================================================================
    Fold-1
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 12s 8ms/step - loss: 0.5897 - accuracy: 0.7810 - val_loss: 0.4856 - val_accuracy: 0.8256
    Epoch 2/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.4231 - accuracy: 0.8456 - val_loss: 0.4166 - val_accuracy: 0.8491
    Epoch 3/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.3750 - accuracy: 0.8621 - val_loss: 0.3945 - val_accuracy: 0.8567
    Epoch 4/32
    1499/1499 [==============================] - 12s 8ms/step - loss: 0.3428 - accuracy: 0.8728 - val_loss: 0.3748 - val_accuracy: 0.8633
    Epoch 5/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.3223 - accuracy: 0.8820 - val_loss: 0.3597 - val_accuracy: 0.8726
    Epoch 6/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.3039 - accuracy: 0.8884 - val_loss: 0.3497 - val_accuracy: 0.8729
    Epoch 7/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2858 - accuracy: 0.8942 - val_loss: 0.3586 - val_accuracy: 0.8694
    Epoch 8/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2715 - accuracy: 0.8989 - val_loss: 0.3334 - val_accuracy: 0.8815
    Epoch 9/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2578 - accuracy: 0.9038 - val_loss: 0.3227 - val_accuracy: 0.8846
    Epoch 10/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2448 - accuracy: 0.9092 - val_loss: 0.3269 - val_accuracy: 0.8833
    Epoch 11/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2334 - accuracy: 0.9132 - val_loss: 0.3248 - val_accuracy: 0.8853
    Epoch 12/32
    1499/1499 [==============================] - 12s 8ms/step - loss: 0.2224 - accuracy: 0.9162 - val_loss: 0.3287 - val_accuracy: 0.8839
    Epoch 13/32
    1499/1499 [==============================] - 11s 8ms/step - loss: 0.2102 - accuracy: 0.9210 - val_loss: 0.3407 - val_accuracy: 0.8826
    Epoch 00013: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3217 - accuracy: 0.8824
    ________________________________________________________________________________
    ================================================================================
    Fold-2
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2386 - accuracy: 0.9124 - val_loss: 0.2157 - val_accuracy: 0.9170
    Epoch 2/32
    1499/1499 [==============================] - 12s 8ms/step - loss: 0.2200 - accuracy: 0.9196 - val_loss: 0.2195 - val_accuracy: 0.9185
    Epoch 3/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2074 - accuracy: 0.9247 - val_loss: 0.2193 - val_accuracy: 0.9177
    Epoch 4/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1979 - accuracy: 0.9279 - val_loss: 0.2237 - val_accuracy: 0.9161
    Epoch 5/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1872 - accuracy: 0.9318 - val_loss: 0.2295 - val_accuracy: 0.9133
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3069 - accuracy: 0.8920
    ________________________________________________________________________________
    ================================================================================
    Fold-3
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1975 - accuracy: 0.9274 - val_loss: 0.1803 - val_accuracy: 0.9326
    Epoch 2/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1823 - accuracy: 0.9324 - val_loss: 0.1838 - val_accuracy: 0.9309
    Epoch 3/32
    1499/1499 [==============================] - 11s 8ms/step - loss: 0.1731 - accuracy: 0.9370 - val_loss: 0.1816 - val_accuracy: 0.9323
    Epoch 4/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1621 - accuracy: 0.9414 - val_loss: 0.1873 - val_accuracy: 0.9311
    Epoch 5/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1547 - accuracy: 0.9427 - val_loss: 0.1982 - val_accuracy: 0.9267
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 2s 5ms/step - loss: 0.3296 - accuracy: 0.8913
    ________________________________________________________________________________
    ================================================================================
    Fold-4
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1683 - accuracy: 0.9392 - val_loss: 0.1413 - val_accuracy: 0.9476
    Epoch 2/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1559 - accuracy: 0.9429 - val_loss: 0.1626 - val_accuracy: 0.9380
    Epoch 3/32
    1499/1499 [==============================] - 12s 8ms/step - loss: 0.1447 - accuracy: 0.9480 - val_loss: 0.1504 - val_accuracy: 0.9448
    Epoch 4/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1353 - accuracy: 0.9517 - val_loss: 0.1671 - val_accuracy: 0.9351
    Epoch 5/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1307 - accuracy: 0.9527 - val_loss: 0.1730 - val_accuracy: 0.9336
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3516 - accuracy: 0.8874
    ________________________________________________________________________________
    ================================================================================
    Fold-5
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 11s 8ms/step - loss: 0.1422 - accuracy: 0.9476 - val_loss: 0.1266 - val_accuracy: 0.9543
    Epoch 2/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1267 - accuracy: 0.9539 - val_loss: 0.1248 - val_accuracy: 0.9541
    Epoch 3/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1192 - accuracy: 0.9571 - val_loss: 0.1396 - val_accuracy: 0.9464
    Epoch 4/32
    1499/1499 [==============================] - 11s 8ms/step - loss: 0.1121 - accuracy: 0.9593 - val_loss: 0.1396 - val_accuracy: 0.9490
    Epoch 5/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1059 - accuracy: 0.9624 - val_loss: 0.1447 - val_accuracy: 0.9474
    Epoch 6/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.0981 - accuracy: 0.9658 - val_loss: 0.1693 - val_accuracy: 0.9367
    Epoch 00006: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3973 - accuracy: 0.8865
    ________________________________________________________________________________

In \[49\]:

    def display_kfold_result(history, k=1):
        # Train & Val Loss
        loss = history.history["loss"]
        val_loss = history.history["val_loss"]
        
        # Train & Val Accuracy
        accuracy = history.history["accuracy"]
        val_accuracy = history.history["val_accuracy"]
        
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(loss, label="Training")
        plt.plot(val_loss, label="Validation")
        plt.legend(loc="upper right")
        
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(accuracy, label="Training")
        plt.plot(val_accuracy, label="Validation")
        plt.legend(loc="lower right")
        
        plt.suptitle("Fold-{}".format(k))
        plt.show()

In \[50\]:

    # Displaying the graph results

    for history in histories:
        display_kfold_result(history, (histories.index(history)+1))

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/a66009d4bdfc2c19c820b94801b2a9f6e9f4bc10.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/5b001e9e2bddb9f4b1fe168e9a44f7b6a944dc3a.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/129ffe2ee9734bf3a66fcaab0076277467b54e78.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/ca986c63315a3671d1cc9697d9456f946641a149.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/f3b9fce0994a26e279f930bd15dd8c1bad753478.png" class="jp-needs-light-background" />

In \[51\]:

    i = 0
    float2 = "{0:.2f}"
    for score in eval_scores:
        percent = score * 100
        print("Fold-{}: {}%".format(i+1, float2.format(percent)))
        i = i + 1

    Fold-1: 88.24%
    Fold-2: 89.20%
    Fold-3: 89.13%
    Fold-4: 88.74%
    Fold-5: 88.65%

## Trying RMSProp Optimizer<a href="#Trying-RMSProp-Optimizer" class="anchor-link">¶</a>

In \[58\]:

    model_2 = lenet5(input_shape, tf.keras.optimizers.RMSprop(learning_rate=0.001))

    Model: "sequential_4"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    conv2d_8 (Conv2D)            (None, 24, 24, 6)         156       
    _________________________________________________________________
    average_pooling2d_8 (Average (None, 12, 12, 6)         0         
    _________________________________________________________________
    conv2d_9 (Conv2D)            (None, 8, 8, 16)          2416      
    _________________________________________________________________
    average_pooling2d_9 (Average (None, 4, 4, 16)          0         
    _________________________________________________________________
    flatten_4 (Flatten)          (None, 256)               0         
    _________________________________________________________________
    dense_12 (Dense)             (None, 120)               30840     
    _________________________________________________________________
    dense_13 (Dense)             (None, 84)                10164     
    _________________________________________________________________
    dense_14 (Dense)             (None, 10)                850       
    =================================================================
    Total params: 44,426
    Trainable params: 44,426
    Non-trainable params: 0
    _________________________________________________________________

In \[59\]:

    from sklearn.model_selection import KFold
    from sklearn.metrics import confusion_matrix
    k = 5
    cross_val = KFold(k, shuffle=True, random_state=1)
    fold_count = 1

    # For training epochs
    epochs = 32

    # For loss & acc plotting
    histories = []

    # For testing/evaluation acc scores
    eval_scores = []

    # For callbacks
    es_callbacks = EarlyStopping(monitor="val_loss",
                                              mode="min",
                                              verbose=1,
                                              patience=4)

In \[60\]:

    for train, validation in cross_val.split(X_train):
        print("="*80)
        print("Fold-{}".format(fold_count))
        print("-"*80)
        print("Training & Validation")
        fold_count = fold_count + 1
        
        
        X_train_, y_train_ = X_train[train], y_train[train]
        X_val, y_val = X_train[validation], y_train[validation]
        
        history = model_2.fit(X_train_, y_train_,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            callbacks=[es_callbacks])
        
        print("-"*80)
        print("Testing/evaluation")
        eval_loss, eval_accuracy = model_2.evaluate(X_test, y_test)
        
        histories.append(history)
        eval_scores.append(eval_accuracy)
        print("_"*80)

    ================================================================================
    Fold-1
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.5779 - accuracy: 0.7847 - val_loss: 0.4955 - val_accuracy: 0.8113
    Epoch 2/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4146 - accuracy: 0.8463 - val_loss: 0.4108 - val_accuracy: 0.8567
    Epoch 3/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3677 - accuracy: 0.8647 - val_loss: 0.3741 - val_accuracy: 0.8655
    Epoch 4/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3381 - accuracy: 0.8768 - val_loss: 0.3776 - val_accuracy: 0.8661
    Epoch 5/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.3176 - accuracy: 0.8830 - val_loss: 0.3453 - val_accuracy: 0.8777
    Epoch 6/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2974 - accuracy: 0.8904 - val_loss: 0.3346 - val_accuracy: 0.8805
    Epoch 7/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2823 - accuracy: 0.8951 - val_loss: 0.3381 - val_accuracy: 0.8755
    Epoch 8/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2678 - accuracy: 0.9001 - val_loss: 0.3313 - val_accuracy: 0.8863
    Epoch 9/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.2562 - accuracy: 0.9036 - val_loss: 0.3167 - val_accuracy: 0.8863
    Epoch 10/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2449 - accuracy: 0.9088 - val_loss: 0.3228 - val_accuracy: 0.8862
    Epoch 11/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2365 - accuracy: 0.9116 - val_loss: 0.3152 - val_accuracy: 0.8892
    Epoch 12/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2259 - accuracy: 0.9166 - val_loss: 0.3393 - val_accuracy: 0.8839
    Epoch 13/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2162 - accuracy: 0.9200 - val_loss: 0.3115 - val_accuracy: 0.8916
    Epoch 14/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2067 - accuracy: 0.9239 - val_loss: 0.3176 - val_accuracy: 0.8924
    Epoch 15/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1993 - accuracy: 0.9262 - val_loss: 0.3212 - val_accuracy: 0.8898
    Epoch 16/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1919 - accuracy: 0.9286 - val_loss: 0.3269 - val_accuracy: 0.8879
    Epoch 17/32
    1499/1499 [==============================] - 11s 8ms/step - loss: 0.1842 - accuracy: 0.9322 - val_loss: 0.3182 - val_accuracy: 0.8928
    Epoch 00017: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3125 - accuracy: 0.8925
    ________________________________________________________________________________
    ================================================================================
    Fold-2
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.2171 - accuracy: 0.9206 - val_loss: 0.1968 - val_accuracy: 0.9263
    Epoch 2/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.2020 - accuracy: 0.9260 - val_loss: 0.1981 - val_accuracy: 0.9280
    Epoch 3/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1906 - accuracy: 0.9305 - val_loss: 0.2011 - val_accuracy: 0.9241
    Epoch 4/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1808 - accuracy: 0.9338 - val_loss: 0.2089 - val_accuracy: 0.9243
    Epoch 5/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1734 - accuracy: 0.9368 - val_loss: 0.2149 - val_accuracy: 0.9234
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3118 - accuracy: 0.8957
    ________________________________________________________________________________
    ================================================================================
    Fold-3
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1841 - accuracy: 0.9331 - val_loss: 0.1499 - val_accuracy: 0.9430
    Epoch 2/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.1717 - accuracy: 0.9377 - val_loss: 0.1705 - val_accuracy: 0.9353
    Epoch 3/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1647 - accuracy: 0.9401 - val_loss: 0.1661 - val_accuracy: 0.9379
    Epoch 4/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1564 - accuracy: 0.9433 - val_loss: 0.1735 - val_accuracy: 0.9347
    Epoch 5/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.1475 - accuracy: 0.9468 - val_loss: 0.1861 - val_accuracy: 0.9304
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3366 - accuracy: 0.8891
    ________________________________________________________________________________
    ================================================================================
    Fold-4
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1590 - accuracy: 0.9420 - val_loss: 0.1248 - val_accuracy: 0.9556
    Epoch 2/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1489 - accuracy: 0.9448 - val_loss: 0.1531 - val_accuracy: 0.9426
    Epoch 3/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1408 - accuracy: 0.9495 - val_loss: 0.1490 - val_accuracy: 0.9430
    Epoch 4/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1331 - accuracy: 0.9532 - val_loss: 0.1632 - val_accuracy: 0.9380
    Epoch 5/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1268 - accuracy: 0.9547 - val_loss: 0.1690 - val_accuracy: 0.9383
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3509 - accuracy: 0.8905
    ________________________________________________________________________________
    ================================================================================
    Fold-5
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1368 - accuracy: 0.9500 - val_loss: 0.1149 - val_accuracy: 0.9577
    Epoch 2/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.1267 - accuracy: 0.9543 - val_loss: 0.1272 - val_accuracy: 0.9531
    Epoch 3/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1198 - accuracy: 0.9563 - val_loss: 0.1278 - val_accuracy: 0.9523
    Epoch 4/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.1140 - accuracy: 0.9589 - val_loss: 0.1448 - val_accuracy: 0.9483
    Epoch 5/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.1071 - accuracy: 0.9609 - val_loss: 0.1462 - val_accuracy: 0.9479
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3914 - accuracy: 0.8864
    ________________________________________________________________________________

In \[55\]:

    def display_kfold_result(history, k=1):
        # Train & Val Loss
        loss = history.history["loss"]
        val_loss = history.history["val_loss"]
        
        # Train & Val Accuracy
        accuracy = history.history["accuracy"]
        val_accuracy = history.history["val_accuracy"]
        
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(loss, label="Training")
        plt.plot(val_loss, label="Validation")
        plt.legend(loc="upper right")
        
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(accuracy, label="Training")
        plt.plot(val_accuracy, label="Validation")
        plt.legend(loc="lower right")
        
        plt.suptitle("Fold-{}".format(k))
        plt.show()

In \[56\]:

    # Displaying the graph results

    for history in histories:
        display_kfold_result(history, (histories.index(history)+1))

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/606a8f78abffb6d2f9520ae77764866ec986197a.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/a0f29ca1ba1c4d17ea4899270359594094e7ec60.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/4dc831bc683352ab56fe30c75efc199e3e125194.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/4daaf2800731350aeca3121468f5d6cd4301f8d0.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/b56e1330c48affb84178ac0b927a5098018ad3a6.png" class="jp-needs-light-background" />

In \[57\]:

    i = 0
    float2 = "{0:.2f}"
    for score in eval_scores:
        percent = score * 100
        print("Fold-{}: {}%".format(i+1, float2.format(percent)))
        i = i + 1

    Fold-1: 88.65%
    Fold-2: 88.46%
    Fold-3: 88.68%
    Fold-4: 88.02%
    Fold-5: 88.34%

## Trying SGD Optimizer<a href="#Trying-SGD-Optimizer" class="anchor-link">¶</a>

In \[61\]:

    model_3 = lenet5(input_shape, tf.keras.optimizers.SGD(learning_rate=0.001))

    Model: "sequential_5"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    conv2d_10 (Conv2D)           (None, 24, 24, 6)         156       
    _________________________________________________________________
    average_pooling2d_10 (Averag (None, 12, 12, 6)         0         
    _________________________________________________________________
    conv2d_11 (Conv2D)           (None, 8, 8, 16)          2416      
    _________________________________________________________________
    average_pooling2d_11 (Averag (None, 4, 4, 16)          0         
    _________________________________________________________________
    flatten_5 (Flatten)          (None, 256)               0         
    _________________________________________________________________
    dense_15 (Dense)             (None, 120)               30840     
    _________________________________________________________________
    dense_16 (Dense)             (None, 84)                10164     
    _________________________________________________________________
    dense_17 (Dense)             (None, 10)                850       
    =================================================================
    Total params: 44,426
    Trainable params: 44,426
    Non-trainable params: 0
    _________________________________________________________________

In \[62\]:

    from sklearn.model_selection import KFold
    from sklearn.metrics import confusion_matrix
    k = 5
    cross_val = KFold(k, shuffle=True, random_state=1)
    fold_count = 1

    # For training epochs
    epochs = 32

    # For loss & acc plotting
    histories = []

    # For testing/evaluation acc scores
    eval_scores = []

    # For callbacks
    es_callbacks = EarlyStopping(monitor="val_loss",
                                              mode="min",
                                              verbose=1,
                                              patience=4)

In \[63\]:

    for train, validation in cross_val.split(X_train):
        print("="*80)
        print("Fold-{}".format(fold_count))
        print("-"*80)
        print("Training & Validation")
        fold_count = fold_count + 1
        
        
        X_train_, y_train_ = X_train[train], y_train[train]
        X_val, y_val = X_train[validation], y_train[validation]
        
        history = model_3.fit(X_train_, y_train_,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            callbacks=[es_callbacks])
        
        print("-"*80)
        print("Testing/evaluation")
        eval_loss, eval_accuracy = model_3.evaluate(X_test, y_test)
        
        histories.append(history)
        eval_scores.append(eval_accuracy)
        print("_"*80)

    ================================================================================
    Fold-1
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 1.8003 - accuracy: 0.4500 - val_loss: 1.3186 - val_accuracy: 0.6237
    Epoch 2/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 1.1062 - accuracy: 0.6729 - val_loss: 0.9928 - val_accuracy: 0.6818
    Epoch 3/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.9144 - accuracy: 0.7028 - val_loss: 0.8777 - val_accuracy: 0.6963
    Epoch 4/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.8241 - accuracy: 0.7171 - val_loss: 0.8088 - val_accuracy: 0.7146
    Epoch 5/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.7665 - accuracy: 0.7286 - val_loss: 0.7633 - val_accuracy: 0.7233
    Epoch 6/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.7264 - accuracy: 0.7363 - val_loss: 0.7299 - val_accuracy: 0.7279
    Epoch 7/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.6970 - accuracy: 0.7435 - val_loss: 0.7070 - val_accuracy: 0.7351
    Epoch 8/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.6743 - accuracy: 0.7499 - val_loss: 0.6858 - val_accuracy: 0.7401
    Epoch 9/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.6559 - accuracy: 0.7559 - val_loss: 0.6693 - val_accuracy: 0.7477
    Epoch 10/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.6402 - accuracy: 0.7606 - val_loss: 0.6557 - val_accuracy: 0.7524
    Epoch 11/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.6265 - accuracy: 0.7659 - val_loss: 0.6426 - val_accuracy: 0.7566
    Epoch 12/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.6145 - accuracy: 0.7704 - val_loss: 0.6314 - val_accuracy: 0.7595
    Epoch 13/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.6036 - accuracy: 0.7755 - val_loss: 0.6211 - val_accuracy: 0.7658
    Epoch 14/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.5939 - accuracy: 0.7794 - val_loss: 0.6113 - val_accuracy: 0.7700
    Epoch 15/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.5848 - accuracy: 0.7831 - val_loss: 0.6022 - val_accuracy: 0.7727
    Epoch 16/32
    1499/1499 [==============================] - 11s 7ms/step - loss: 0.5765 - accuracy: 0.7867 - val_loss: 0.5948 - val_accuracy: 0.7727
    Epoch 17/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.5688 - accuracy: 0.7900 - val_loss: 0.5873 - val_accuracy: 0.7770
    Epoch 18/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5616 - accuracy: 0.7923 - val_loss: 0.5804 - val_accuracy: 0.7821
    Epoch 19/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.5550 - accuracy: 0.7947 - val_loss: 0.5739 - val_accuracy: 0.7839
    Epoch 20/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5487 - accuracy: 0.7976 - val_loss: 0.5668 - val_accuracy: 0.7879
    Epoch 21/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5427 - accuracy: 0.8012 - val_loss: 0.5627 - val_accuracy: 0.7882
    Epoch 22/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.5370 - accuracy: 0.8036 - val_loss: 0.5559 - val_accuracy: 0.7891
    Epoch 23/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5320 - accuracy: 0.8052 - val_loss: 0.5500 - val_accuracy: 0.7924
    Epoch 24/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.5268 - accuracy: 0.8070 - val_loss: 0.5454 - val_accuracy: 0.7919
    Epoch 25/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.5222 - accuracy: 0.8089 - val_loss: 0.5413 - val_accuracy: 0.7996
    Epoch 26/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.5172 - accuracy: 0.8117 - val_loss: 0.5354 - val_accuracy: 0.7994
    Epoch 27/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5131 - accuracy: 0.8135 - val_loss: 0.5320 - val_accuracy: 0.8022
    Epoch 28/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5088 - accuracy: 0.8141 - val_loss: 0.5263 - val_accuracy: 0.8040
    Epoch 29/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.5047 - accuracy: 0.8164 - val_loss: 0.5216 - val_accuracy: 0.8058
    Epoch 30/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.5005 - accuracy: 0.8182 - val_loss: 0.5183 - val_accuracy: 0.8077
    Epoch 31/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4966 - accuracy: 0.8207 - val_loss: 0.5143 - val_accuracy: 0.8078
    Epoch 32/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4931 - accuracy: 0.8219 - val_loss: 0.5098 - val_accuracy: 0.8105
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.4950 - accuracy: 0.8223
    ________________________________________________________________________________
    ================================================================================
    Fold-2
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4922 - accuracy: 0.8213 - val_loss: 0.4950 - val_accuracy: 0.8230
    Epoch 2/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4883 - accuracy: 0.8230 - val_loss: 0.4917 - val_accuracy: 0.8240
    Epoch 3/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4846 - accuracy: 0.8244 - val_loss: 0.4907 - val_accuracy: 0.8230
    Epoch 4/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4810 - accuracy: 0.8251 - val_loss: 0.4865 - val_accuracy: 0.8271
    Epoch 5/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4775 - accuracy: 0.8271 - val_loss: 0.4841 - val_accuracy: 0.8280
    Epoch 6/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4743 - accuracy: 0.8285 - val_loss: 0.4808 - val_accuracy: 0.8290
    Epoch 7/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4709 - accuracy: 0.8300 - val_loss: 0.4775 - val_accuracy: 0.8295
    Epoch 8/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4679 - accuracy: 0.8307 - val_loss: 0.4744 - val_accuracy: 0.8296
    Epoch 9/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4647 - accuracy: 0.8321 - val_loss: 0.4729 - val_accuracy: 0.8317
    Epoch 10/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4616 - accuracy: 0.8336 - val_loss: 0.4694 - val_accuracy: 0.8336
    Epoch 11/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4586 - accuracy: 0.8339 - val_loss: 0.4706 - val_accuracy: 0.8311
    Epoch 12/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4559 - accuracy: 0.8358 - val_loss: 0.4646 - val_accuracy: 0.8344
    Epoch 13/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4533 - accuracy: 0.8365 - val_loss: 0.4628 - val_accuracy: 0.8336
    Epoch 14/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4506 - accuracy: 0.8384 - val_loss: 0.4605 - val_accuracy: 0.8357
    Epoch 15/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4480 - accuracy: 0.8385 - val_loss: 0.4570 - val_accuracy: 0.8381
    Epoch 16/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4453 - accuracy: 0.8400 - val_loss: 0.4552 - val_accuracy: 0.8385
    Epoch 17/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4428 - accuracy: 0.8406 - val_loss: 0.4541 - val_accuracy: 0.8372
    Epoch 18/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4405 - accuracy: 0.8417 - val_loss: 0.4524 - val_accuracy: 0.8383
    Epoch 19/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4382 - accuracy: 0.8426 - val_loss: 0.4499 - val_accuracy: 0.8397
    Epoch 20/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4360 - accuracy: 0.8427 - val_loss: 0.4480 - val_accuracy: 0.8421
    Epoch 21/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4336 - accuracy: 0.8447 - val_loss: 0.4445 - val_accuracy: 0.8421
    Epoch 22/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4315 - accuracy: 0.8445 - val_loss: 0.4446 - val_accuracy: 0.8412
    Epoch 23/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4294 - accuracy: 0.8466 - val_loss: 0.4409 - val_accuracy: 0.8444
    Epoch 24/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4274 - accuracy: 0.8463 - val_loss: 0.4407 - val_accuracy: 0.8438
    Epoch 25/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4254 - accuracy: 0.8470 - val_loss: 0.4380 - val_accuracy: 0.8454
    Epoch 26/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4235 - accuracy: 0.8486 - val_loss: 0.4364 - val_accuracy: 0.8452
    Epoch 27/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4217 - accuracy: 0.8482 - val_loss: 0.4347 - val_accuracy: 0.8455
    Epoch 28/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4197 - accuracy: 0.8486 - val_loss: 0.4333 - val_accuracy: 0.8469
    Epoch 29/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4178 - accuracy: 0.8491 - val_loss: 0.4310 - val_accuracy: 0.8481
    Epoch 30/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4162 - accuracy: 0.8500 - val_loss: 0.4306 - val_accuracy: 0.8475
    Epoch 31/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4143 - accuracy: 0.8502 - val_loss: 0.4276 - val_accuracy: 0.8489
    Epoch 32/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4125 - accuracy: 0.8516 - val_loss: 0.4265 - val_accuracy: 0.8490
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.4152 - accuracy: 0.8553
    ________________________________________________________________________________
    ================================================================================
    Fold-3
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4165 - accuracy: 0.8501 - val_loss: 0.4020 - val_accuracy: 0.8576
    Epoch 2/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4145 - accuracy: 0.8507 - val_loss: 0.4025 - val_accuracy: 0.8566
    Epoch 3/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4129 - accuracy: 0.8505 - val_loss: 0.4001 - val_accuracy: 0.8591
    Epoch 4/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4111 - accuracy: 0.8509 - val_loss: 0.3991 - val_accuracy: 0.8584
    Epoch 5/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4094 - accuracy: 0.8519 - val_loss: 0.3976 - val_accuracy: 0.8601
    Epoch 6/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4078 - accuracy: 0.8528 - val_loss: 0.3962 - val_accuracy: 0.8606
    Epoch 7/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.4061 - accuracy: 0.8532 - val_loss: 0.3957 - val_accuracy: 0.8606
    Epoch 8/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4047 - accuracy: 0.8538 - val_loss: 0.3941 - val_accuracy: 0.8605
    Epoch 9/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4032 - accuracy: 0.8550 - val_loss: 0.3933 - val_accuracy: 0.8611
    Epoch 10/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.4017 - accuracy: 0.8555 - val_loss: 0.3937 - val_accuracy: 0.8604
    Epoch 11/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.4003 - accuracy: 0.8556 - val_loss: 0.3916 - val_accuracy: 0.8611
    Epoch 12/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3986 - accuracy: 0.8556 - val_loss: 0.3906 - val_accuracy: 0.8610
    Epoch 13/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3976 - accuracy: 0.8563 - val_loss: 0.3894 - val_accuracy: 0.8626
    Epoch 14/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3959 - accuracy: 0.8571 - val_loss: 0.3883 - val_accuracy: 0.8621
    Epoch 15/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3945 - accuracy: 0.8570 - val_loss: 0.3880 - val_accuracy: 0.8618
    Epoch 16/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3933 - accuracy: 0.8580 - val_loss: 0.3865 - val_accuracy: 0.8642
    Epoch 17/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3919 - accuracy: 0.8584 - val_loss: 0.3851 - val_accuracy: 0.8646
    Epoch 18/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3906 - accuracy: 0.8588 - val_loss: 0.3837 - val_accuracy: 0.8649
    Epoch 19/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3891 - accuracy: 0.8592 - val_loss: 0.3836 - val_accuracy: 0.8636
    Epoch 20/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3878 - accuracy: 0.8593 - val_loss: 0.3831 - val_accuracy: 0.8650
    Epoch 21/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3865 - accuracy: 0.8607 - val_loss: 0.3826 - val_accuracy: 0.8649
    Epoch 22/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3854 - accuracy: 0.8605 - val_loss: 0.3801 - val_accuracy: 0.8664
    Epoch 23/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3842 - accuracy: 0.8611 - val_loss: 0.3800 - val_accuracy: 0.8664
    Epoch 24/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3829 - accuracy: 0.8613 - val_loss: 0.3777 - val_accuracy: 0.8673
    Epoch 25/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3818 - accuracy: 0.8615 - val_loss: 0.3774 - val_accuracy: 0.8679
    Epoch 26/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3807 - accuracy: 0.8625 - val_loss: 0.3774 - val_accuracy: 0.8664
    Epoch 27/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3794 - accuracy: 0.8625 - val_loss: 0.3783 - val_accuracy: 0.8653
    Epoch 28/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3782 - accuracy: 0.8631 - val_loss: 0.3761 - val_accuracy: 0.8688
    Epoch 29/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3772 - accuracy: 0.8638 - val_loss: 0.3751 - val_accuracy: 0.8647
    Epoch 30/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3760 - accuracy: 0.8641 - val_loss: 0.3734 - val_accuracy: 0.8687
    Epoch 31/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3750 - accuracy: 0.8640 - val_loss: 0.3723 - val_accuracy: 0.8693
    Epoch 32/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3738 - accuracy: 0.8648 - val_loss: 0.3716 - val_accuracy: 0.8694
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3794 - accuracy: 0.8640
    ________________________________________________________________________________
    ================================================================================
    Fold-4
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3757 - accuracy: 0.8665 - val_loss: 0.3585 - val_accuracy: 0.8674
    Epoch 2/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3744 - accuracy: 0.8666 - val_loss: 0.3590 - val_accuracy: 0.8670
    Epoch 3/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3732 - accuracy: 0.8670 - val_loss: 0.3581 - val_accuracy: 0.8681
    Epoch 4/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3721 - accuracy: 0.8674 - val_loss: 0.3593 - val_accuracy: 0.8660
    Epoch 5/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3712 - accuracy: 0.8672 - val_loss: 0.3562 - val_accuracy: 0.8682
    Epoch 6/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3699 - accuracy: 0.8682 - val_loss: 0.3564 - val_accuracy: 0.8680
    Epoch 7/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3690 - accuracy: 0.8678 - val_loss: 0.3566 - val_accuracy: 0.8697
    Epoch 8/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3677 - accuracy: 0.8688 - val_loss: 0.3566 - val_accuracy: 0.8674
    Epoch 9/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3670 - accuracy: 0.8691 - val_loss: 0.3545 - val_accuracy: 0.8682
    Epoch 10/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3658 - accuracy: 0.8691 - val_loss: 0.3538 - val_accuracy: 0.8692
    Epoch 11/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3645 - accuracy: 0.8701 - val_loss: 0.3527 - val_accuracy: 0.8696
    Epoch 12/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3638 - accuracy: 0.8700 - val_loss: 0.3518 - val_accuracy: 0.8695
    Epoch 13/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3626 - accuracy: 0.8706 - val_loss: 0.3517 - val_accuracy: 0.8696
    Epoch 14/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3619 - accuracy: 0.8707 - val_loss: 0.3517 - val_accuracy: 0.8687
    Epoch 15/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3607 - accuracy: 0.8717 - val_loss: 0.3530 - val_accuracy: 0.8680
    Epoch 16/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3600 - accuracy: 0.8712 - val_loss: 0.3492 - val_accuracy: 0.8727
    Epoch 17/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3589 - accuracy: 0.8715 - val_loss: 0.3482 - val_accuracy: 0.8702
    Epoch 18/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3578 - accuracy: 0.8715 - val_loss: 0.3488 - val_accuracy: 0.8708
    Epoch 19/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3570 - accuracy: 0.8724 - val_loss: 0.3484 - val_accuracy: 0.8707
    Epoch 20/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3561 - accuracy: 0.8731 - val_loss: 0.3463 - val_accuracy: 0.8712
    Epoch 21/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3552 - accuracy: 0.8725 - val_loss: 0.3474 - val_accuracy: 0.8720
    Epoch 22/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3541 - accuracy: 0.8738 - val_loss: 0.3459 - val_accuracy: 0.8720
    Epoch 23/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3535 - accuracy: 0.8742 - val_loss: 0.3445 - val_accuracy: 0.8720
    Epoch 24/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3525 - accuracy: 0.8744 - val_loss: 0.3449 - val_accuracy: 0.8702
    Epoch 25/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3515 - accuracy: 0.8738 - val_loss: 0.3448 - val_accuracy: 0.8714
    Epoch 26/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3509 - accuracy: 0.8745 - val_loss: 0.3428 - val_accuracy: 0.8727
    Epoch 27/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3499 - accuracy: 0.8755 - val_loss: 0.3427 - val_accuracy: 0.8735
    Epoch 28/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3487 - accuracy: 0.8764 - val_loss: 0.3438 - val_accuracy: 0.8730
    Epoch 29/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3479 - accuracy: 0.8766 - val_loss: 0.3421 - val_accuracy: 0.8729
    Epoch 30/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3471 - accuracy: 0.8760 - val_loss: 0.3405 - val_accuracy: 0.8743
    Epoch 31/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3464 - accuracy: 0.8767 - val_loss: 0.3399 - val_accuracy: 0.8736
    Epoch 32/32
    1499/1499 [==============================] - 9s 6ms/step - loss: 0.3456 - accuracy: 0.8773 - val_loss: 0.3388 - val_accuracy: 0.8741
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3562 - accuracy: 0.8693
    ________________________________________________________________________________
    ================================================================================
    Fold-5
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3430 - accuracy: 0.8768 - val_loss: 0.3456 - val_accuracy: 0.8782
    Epoch 2/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3420 - accuracy: 0.8772 - val_loss: 0.3473 - val_accuracy: 0.8757
    Epoch 3/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3410 - accuracy: 0.8777 - val_loss: 0.3469 - val_accuracy: 0.8766
    Epoch 4/32
    1499/1499 [==============================] - 10s 6ms/step - loss: 0.3401 - accuracy: 0.8776 - val_loss: 0.3473 - val_accuracy: 0.8754
    Epoch 5/32
    1499/1499 [==============================] - 10s 7ms/step - loss: 0.3391 - accuracy: 0.8784 - val_loss: 0.3456 - val_accuracy: 0.8776
    Epoch 00005: early stopping
    --------------------------------------------------------------------------------
    Testing/evaluation
    313/313 [==============================] - 1s 3ms/step - loss: 0.3524 - accuracy: 0.8696
    ________________________________________________________________________________

In \[64\]:

    def display_kfold_result(history, k=1):
        # Train & Val Loss
        loss = history.history["loss"]
        val_loss = history.history["val_loss"]
        
        # Train & Val Accuracy
        accuracy = history.history["accuracy"]
        val_accuracy = history.history["val_accuracy"]
        
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(loss, label="Training")
        plt.plot(val_loss, label="Validation")
        plt.legend(loc="upper right")
        
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(accuracy, label="Training")
        plt.plot(val_accuracy, label="Validation")
        plt.legend(loc="lower right")
        
        plt.suptitle("Fold-{}".format(k))
        plt.show()

In \[65\]:

    # Displaying the graph results

    for history in histories:
        display_kfold_result(history, (histories.index(history)+1))

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/51cdbbcbafcd3f40bb2e3ffeec2a99c816751142.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/3628fb97a2ed600f8952f6954dc1e6c038f1dfd1.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/4f95a22e0b22ce35841c79756f50a767d4241009.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/d4ff03591c62e6f6487bde3898678f841f583b93.png" class="jp-needs-light-background" />

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/53093ad823a0a63e07321a0b1ddf0ad8950056c2.png" class="jp-needs-light-background" />

In \[66\]:

    i = 0
    float2 = "{0:.2f}"
    for score in eval_scores:
        percent = score * 100
        print("Fold-{}: {}%".format(i+1, float2.format(percent)))
        i = i + 1

    Fold-1: 82.23%
    Fold-2: 85.53%
    Fold-3: 86.40%
    Fold-4: 86.93%
    Fold-5: 86.96%

# Transfer Learning<a href="#Transfer-Learning" class="anchor-link">¶</a>

## Trying to use another 2 different CNNs models to compare the results with LeNet5-5<a href="#Trying-to-use-another-2-different-CNNs-models-to-compare-the-results-with-LeNet5-5" class="anchor-link">¶</a>

### Implementing functions for resizing the images and making it with 3 channels to be suitable with the other models<a href="#Implementing-functions-for-resizing-the-images-and-making-it-with-3-channels-to-be-suitable-with-the-other-models" class="anchor-link">¶</a>

In \[67\]:

    import cv2
    def to_3_channels(X_1):
        X_3 = np.empty([X_1.shape[0], X_1.shape[1], X_1.shape[2], 3], dtype=float)

        for i in range(X_1.shape[0]):
            X_3[i] = cv2.merge([X_1[i],X_1[i],X_1[i]])
            
        return X_3

In \[68\]:

    def resize_image(X_3, x_size, y_size):
        X_3_resized = np.empty([X_3.shape[0], x_size, y_size, 3], dtype=float)

        for i in range(X_3.shape[0]):
            X_3_resized[i] = tf.image.resize(X_3[i], (x_size, y_size))
            
        return X_3_resized

### Transforming Images<a href="#Transforming-Images" class="anchor-link">¶</a>

In \[69\]:

    X_train_3_32 = resize_image(to_3_channels(X_train), 32, 32)
    X_test_3_32 = resize_image(to_3_channels(X_test), 32, 32)

In \[70\]:

    print(X_train_3_32.shape, X_train.shape)
    print(X_test_3_32.shape, X_test.shape)

    (59957, 32, 32, 3) (59957, 28, 28, 1)
    (10000, 32, 32, 3) (10000, 28, 28, 1)

In \[71\]:

    print(X_train.shape, to_3_channels(X_train).shape)
    print(type(X_train), type(to_3_channels(X_train)))
    print(X_train[0][0][0], to_3_channels(X_train)[0][0][0], to_3_channels(X_train)[0][0][0][0])

    plt.subplot(1, 2, 1)
    plt.imshow(256*X_train[1], cmap='gray', vmin=0, vmax=255)
    #plt.legend(loc="left")

    plt.subplot(1, 2, 2)
    plt.imshow(X_train_3_32[1], cmap='brg', vmin=0, vmax=255)
    #plt.legend(loc="right")
    plt.show()

    (59957, 28, 28, 1) (59957, 28, 28, 3)
    <class 'numpy.ndarray'> <class 'numpy.ndarray'>
    [0.] [0. 0. 0.] 0.0

<img src="attachment:vertopal_f27c1d22bea84e3a859cbade3859e04e/9e91d07bcf13dc12d4d38f65988894b24ac22a9e.png" class="jp-needs-light-background" />

## Trying with VGG16<a href="#Trying-with-VGG16" class="anchor-link">¶</a>

In \[74\]:

    from keras.models import Model

In \[77\]:

    from keras.applications.vgg16 import VGG16
    from keras.layers import Dense, Flatten

    model_VGG16 = VGG16(include_top=False, input_shape=(32,32,3), weights='imagenet')

    # Freeze all the layers
    for layer in model_VGG16.layers[:]:
        layer.trainable = False

    # Add Dense layer as in VGG16
    output = model_VGG16.output
    output = Flatten()(output)
    output = Dense(units=100, activation='relu')(output)
    output = Dense(units=10, activation='softmax')(output)
    model_VGG16 = Model(model_VGG16.input, output)
    model_VGG16.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    model_VGG16.summary()

    Model: "model_2"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
    _________________________________________________________________
    block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
    _________________________________________________________________
    block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
    _________________________________________________________________
    block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
    _________________________________________________________________
    block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
    _________________________________________________________________
    block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
    _________________________________________________________________
    block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
    _________________________________________________________________
    block3_conv1 (Conv2D)        (None, 8, 8, 256)         295168    
    _________________________________________________________________
    block3_conv2 (Conv2D)        (None, 8, 8, 256)         590080    
    _________________________________________________________________
    block3_conv3 (Conv2D)        (None, 8, 8, 256)         590080    
    _________________________________________________________________
    block3_pool (MaxPooling2D)   (None, 4, 4, 256)         0         
    _________________________________________________________________
    block4_conv1 (Conv2D)        (None, 4, 4, 512)         1180160   
    _________________________________________________________________
    block4_conv2 (Conv2D)        (None, 4, 4, 512)         2359808   
    _________________________________________________________________
    block4_conv3 (Conv2D)        (None, 4, 4, 512)         2359808   
    _________________________________________________________________
    block4_pool (MaxPooling2D)   (None, 2, 2, 512)         0         
    _________________________________________________________________
    block5_conv1 (Conv2D)        (None, 2, 2, 512)         2359808   
    _________________________________________________________________
    block5_conv2 (Conv2D)        (None, 2, 2, 512)         2359808   
    _________________________________________________________________
    block5_conv3 (Conv2D)        (None, 2, 2, 512)         2359808   
    _________________________________________________________________
    block5_pool (MaxPooling2D)   (None, 1, 1, 512)         0         
    _________________________________________________________________
    flatten_9 (Flatten)          (None, 512)               0         
    _________________________________________________________________
    dense_24 (Dense)             (None, 100)               51300     
    _________________________________________________________________
    dense_25 (Dense)             (None, 10)                1010      
    =================================================================
    Total params: 14,766,998
    Trainable params: 52,310
    Non-trainable params: 14,714,688
    _________________________________________________________________

In \[78\]:

    from sklearn.model_selection import KFold
    from sklearn.metrics import confusion_matrix
    k = 5
    cross_val = KFold(k, shuffle=True, random_state=1)
    fold_count = 1

    # For training epochs
    epochs = 15

    # For loss & acc plotting
    histories = []

    # For testing/evaluation acc scores
    eval_scores = []

    # For callbacks
    es_callbacks = EarlyStopping(monitor="val_loss",
                                              mode="min",
                                              verbose=1,
                                              patience=4)

In \[ \]:

    for train, validation in cross_val.split(X_train):
        print("="*80)
        print("Fold-{}".format(fold_count))
        print("-"*80)
        print("Training & Validation")
        fold_count = fold_count + 1
        
        
        X_train_, y_train_ = X_train_3_32[train], y_train[train]
        X_val, y_val = X_train_3_32[validation], y_train[validation]
        
        history = model_VGG16.fit(X_train_, y_train_,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            callbacks=[es_callbacks])
        
        print("-"*80)
        print("Testing/evaluation")
        eval_loss, eval_accuracy = model_VGG16.evaluate(X_test_3_32, y_test)
        
        histories.append(history)
        eval_scores.append(eval_accuracy)
        print("_"*80)

    ================================================================================
    Fold-1
    --------------------------------------------------------------------------------
    Training & Validation
    Epoch 1/15
     283/1499 [====>.........................] - ETA: 4:17 - loss: 0.8759 - accuracy: 0.7151

In \[ \]:

    def display_kfold_result(history, k=1):
        # Train & Val Loss
        loss = history.history["loss"]
        val_loss = history.history["val_loss"]
        
        # Train & Val Accuracy
        accuracy = history.history["accuracy"]
        val_accuracy = history.history["val_accuracy"]
        
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(loss, label="Training")
        plt.plot(val_loss, label="Validation")
        plt.legend(loc="upper right")
        
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(accuracy, label="Training")
        plt.plot(val_accuracy, label="Validation")
        plt.legend(loc="lower right")
        
        plt.suptitle("Fold-{}".format(k))
        plt.show()

In \[ \]:

    # Displaying the graph results

    for history in histories:
        display_kfold_result(history, (histories.index(history)+1))

In \[ \]:

    i = 0
    float2 = "{0:.2f}"
    for score in eval_scores:
        percent = score * 100
        print("Fold-{}: {}%".format(i+1, float2.format(percent)))
        i = i + 1

## Trying with ResNet50<a href="#Trying-with-ResNet50" class="anchor-link">¶</a>

In \[ \]:

    from keras.models import Model
    from keras.layers import Dense, GlobalAveragePooling2D
    from tensorflow.keras.applications.resnet50 import ResNet50

    model_RESNET = ResNet50(include_top=False, input_shape=(32,32,3), weights='imagenet')

    # Freeze all the layers
    for layer in model_RESNET.layers[:]:
        layer.trainable = False

    # Add Dense layer to classify on CIFAR10
    output = model_RESNET.output
    output = GlobalAveragePooling2D()(output)
    output = Dense(units=10, activation='softmax')(output)
    model_RESNET = Model(model_RESNET.input, output)
    model_RESNET.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    model_RESNET.summary()

In \[ \]:

    visualkeras.layered_view(model_RESNET)

In \[ \]:

    from sklearn.model_selection import KFold
    from sklearn.metrics import confusion_matrix
    k = 5
    cross_val = KFold(k, shuffle=True, random_state=1)
    fold_count = 1

    # For training epochs
    epochs = 15

    # For loss & acc plotting
    histories = []

    # For testing/evaluation acc scores
    eval_scores = []

    # For callbacks
    es_callbacks = EarlyStopping(monitor="val_loss",
                                              mode="min",
                                              verbose=1,
                                              patience=4)

In \[ \]:

    for train, validation in cross_val.split(X_train):
        print("="*80)
        print("Fold-{}".format(fold_count))
        print("-"*80)
        print("Training & Validation")
        fold_count = fold_count + 1
        
        
        X_train_, y_train_ = X_train_3_32[train], y_train[train]
        X_val, y_val = X_train_3_32[validation], y_train[validation]
        
        history = model_RESNET.fit(X_train_, y_train_,
                            epochs=epochs,
                            validation_data=(X_val, y_val),
                            callbacks=[es_callbacks])
        
        print("-"*80)
        print("Testing/evaluation")
        eval_loss, eval_accuracy = model_RESNET.evaluate(X_test_3_32, y_test)
        
        histories.append(history)
        eval_scores.append(eval_accuracy)
        print("_"*80)

In \[ \]:

    def display_kfold_result(history, k=1):
        # Train & Val Loss
        loss = history.history["loss"]
        val_loss = history.history["val_loss"]
        
        # Train & Val Accuracy
        accuracy = history.history["accuracy"]
        val_accuracy = history.history["val_accuracy"]
        
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(loss, label="Training")
        plt.plot(val_loss, label="Validation")
        plt.legend(loc="upper right")
        
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(accuracy, label="Training")
        plt.plot(val_accuracy, label="Validation")
        plt.legend(loc="lower right")
        
        plt.suptitle("Fold-{}".format(k))
        plt.show()

In \[ \]:

    # Displaying the graph results

    for history in histories:
        display_kfold_result(history, (histories.index(history)+1))

In \[ \]:

    i = 0
    float2 = "{0:.2f}"
    for score in eval_scores:
        percent = score * 100
        print("Fold-{}: {}%".format(i+1, float2.format(percent)))
        i = i + 1

## Comparing the Results of the 3 Models<a href="#Comparing-the-Results-of-the-3-Models" class="anchor-link">¶</a>

In \[ \]:

     

## Comment on why you think LeNet-5 further improves the accuracy if any at all. And if it doesn't, why not?<a href="#Comment-on-why-you-think-LeNet-5-further-improves-the-accuracy-if-any-at-all.-And-if-it-doesn&#39;t,-why-not?" class="anchor-link">¶</a>

### The architecture was designed to identify handwritten digits in the MNIST data-set and wasn't intended for image recognition but still in our case it out performs the other models several probable reasons<a href="#The-architecture-was-designed-to-identify-handwritten-digits-in-the-MNIST-data-set-and-wasn&#39;t-intended-for-image-recognition-but-still-in-our-case-it-out-performs-the-other-models-several-probable-reasons" class="anchor-link">¶</a>

-   We did the hyperparameter tuning and the k-fold evaluation to
    achieve the very best performance we can get, which won't be
    feasible on the other complicated CNNs we used.
-   We used a smaller batch size and a larger number of epochs since the
    architecture is quite easier and simpler and way faster than the
    other complicated CNNs we used.
-   The LeNet is originally intended for grayscale images so it gives a
    somewhat descent performance, while both ResNet50 & VGG16 are
    intended for rgb images and also the images needed to be rescaled
    which is done crudely and that affected the perfrmance